In [1]:
from sage.all import *

# Let's use the same primes

Method : we have a guarantee that after some iterations that gcd of ri's is equal to one after when interacting with the oracle. We compute a linear combination's of ri's such that their gcd = 1 using extended euclidean algorithm.Finally we raise each message to it's corresponding linear combination coefficient and multiply all of them together to get the message.

In [2]:
N1 = 2083942285611508539086265595840010478015109036439363635280587182395374938322559822338668376791455665784726914688183444532432720065787986149494796802755789
k1 = 10000000

In [6]:
import socket

def connect_server(server_name, port, message,exercise_num):
    R,msgs = [],[]
    while True :
        server = (server_name, int(port)) #calling int is required when using Sage
        s = socket.create_connection(server)
        s.send(message + "\n")
        response=""
        while True: #data might come in several packets, need to wait for all of it
            data = s.recv(1024)
            if len(data) == 0:
                break
            if data[-1] == '\n': 
                response += data[:-1]  
                break
            response += data
        s.close()
        if exercise_num == 1:
            response = response.split()
            (r, s) = (Integer(response[0]), Integer(response[1]))
            R.append(r),msgs.append(s)
            if gcd(R) == 1: # keep iterating until the gcd of these elements are equal to one
                break
        elif exercise_num == 4:
            R.append(Integer(response))
            break
        else:
            raise NotImplementedError('You don\'t need the oracle for this exercise')
            
    return R,msgs

In [4]:
sciper = 281141 
R,msgs = connect_server("lasecpc28.epfl.ch", int(5557), str(sciper),1)

In [5]:
# to compute the linear combinations we compute extended euclid algorithm for  the first two numbers
# and then for number_index > 2 run extended euclid on this number and the gcd of previous numbers and get the coefficients
# multiply the coeffient with all previous coefficients
multiples = []
d,u,v = xgcd(R[0],R[1])
multiples = multiples + [u,v]
for i in range(2,len(R)):
    d,u,v = xgcd(d,R[i])
    for j in range(0,len(multiples)):
        multiples[j] *= u
    multiples.append(v)

In [6]:
powered_msgs = []
for i in range(0,len(msgs)):
    power = power_mod(msgs[i],multiples[i],N1) # compute x^(r * u) mod N
    powered_msgs.append(power)

In [7]:
a = mod(1,N1) # extracted msg
for i in range(len(powered_msgs)):
    a = mod(a * (powered_msgs[i]),N1) # multiply all powers together to get the message

In [8]:
a

964428547818887150428295297514716956252811134547063672356929752891152989036750401547695146936543814106188665060062404896055045495815009802146011649735258

# Elements of order 2

to find elements of order 2 in Zn*. We compute x^2 modn = 1 mod n. <br>
By using crt we break it into x^2 mod all factors. <br>
Using crt we compute all elements y = +- a mod (a factor)

In [9]:
p2=12490436598689293019793752498850223593851732009260629039975438903671328146869063182612769347335640257217682882606463941915694214642933253669651242233731493
q2=2642114135812671148107535111615353486904341745399136732978421386826708109392096521387134896639485435398079840862371836234226625310585012674950578452958753
r2=9099939049609086807281872311843106462045192786932332797977808812988238231804818845060172242545014348892888236169925374497726925735063361230882322807183287
s2=10869552176137642414794948789200666476105262377026106603586002905760590166325944222115378404695199692307813462962202498427160195222812918227181532132988331
n2=3264219305070411761717098486784697496575927919380732103339590174941686404677791121188725211613736383949568526009617392833745621411274511491395545285544441664088106155515882558398660399294677479275713971259792435776592750311402407091190452831912038051453215157058845989198673511319078377684555319426464616411715168600642618223004218307736114471406314856768548839984908075583774158781172524815361316468069382720465869729665164144887339975540246423425029023145329604150789324282282652584713580407201260295365164777652704663007142160772444785137830566921751449386087903380826836597619382095266216146395751468473545971313

In [10]:
PMod  = mod(1,p2)
PRoot = PMod.sqrt(all=True)
QMod  = mod(1,q2)
QRoot = QMod.sqrt(all=True)
RMod  = mod(1,r2)
RRoot = RMod.sqrt(all=True)
SMod  = mod(1,s2)
SRoot = SMod.sqrt(all=True)

module = [Integer(p2),Integer(q2),Integer(r2),Integer(s2)]
Root = []
for p in PRoot:
    for q in QRoot:
        for r in RRoot:
            for s in SRoot:
                modlist = [Integer(p),Integer(q),Integer(r),Integer(s)]
                r = CRT_list(modlist,module)
                Root.append(r)

In [11]:
sorted(Root)

[1,
 138580805084190773079229336565482202570141161433761828123405344128473460913288613437117537737387544586954925399101211351722356199081055319265122461042875564110851893566895591838587484221171787377807153008337955470062416768555315882123732694919189636464395844061789349048647919819025775149940042808053290150545073527978412790905774181236345122988780211888347407947606285415020826831211775769391861833573147628228492448516176319776607115750766948265765709184112023573478797135906663751038172334749154064280031651276078757144275424956478058139185132099348632484928320246555434268965510338128308482032456997359994794766,
 49286446094839598064498565444715075084663038306849912540163803970341260591225522940313138129861103740874695348787473318762911386732235923374481011992199775705190912609883502604209390472846759756189114627440704241342684081457066599198512414670468643279423391321734939189106993763326305721684796072217375009361481335473004299509897860492185692808894319391041573997571120051984795

# RSA with subgroup 

method : For this part we know that all cipher texts have the form of g^(e * encoding(ci)) mod N <br>
we know that space must be included in the message <br>
We raise each cipher element in the array to the power 27(maximum encoding) and iterate downwards , whenever this power belongs to the cipher text then this means that we got the encoding of the space element. <br>
then we raise the encoding of the space to the range of powers from 2 untill 27 and create a dictionary mapping every cipher text to it's respective letter in the alphabet <br>

In [12]:
# read rsa modulus
n3 = read_data('./Asst2/281141/281141-parameters.txt',str) 
n3 = Integer(n3[11].split('=')[1].strip())

In [13]:
# read cipher texts
c3 = read_data('./Asst2/281141/281141-parameters.txt',str) 
c3 = c3[12].split('=')[1:][0]
c3 = c3.replace('[','').replace(']','').split(',')

In [14]:
c3 = [Integer(x) for x in c3]

In [15]:
letters = range(1,28) # range of letter encoding

In [16]:
space_encoding = None
for encoding in reversed(letters):
    for cipher_text in c3:
        check_space = power_mod(cipher_text,encoding,n3) # compute g^ e * maximum encoding
        if check_space in c3: # check if this encoding is inside the cipher text s
            space_encoding = cipher_text # if true this is the space encoding
            
    if space_encoding is not None:
        break

In [17]:
encodings = dict({space_encoding:1})
for i in letters:
    if i > 1:
        checker = power_mod(space_encoding,i,n3) # raise each power to the encoding of a letter
        for cipher_text in c3:
            if cipher_text == checker: # check if it is in the cipher text
                encodings[cipher_text] = i # add this mapping to the dictionary
                break

In [18]:
# get plain text by replacing every cipher text encoding in the dictionary with the respective asccii letter
replace = [] 
result = ""
for cipher_text in c3:
    int_encoding = encodings[cipher_text]
    if int_encoding == 1:
        result += ' '
    else:
        result += str(unichr(95 + int_encoding))
print(result)

i do not know how should i do with this sentence it must contain a random word like exempting not to make all answers same for all students


# He who gets the orders knows the factors

Method : for this part we first compute the epxonent of the group ZN^2 and then from the exponent we compute the factor of N^2. Given this factor we compute the gcd of this factor and N , the extracted value is one of N's two factors and thus we extract p and q and output the larger one.

In [7]:
# read N the group modulus
c4 = read_data('./Asst2/281141/281141-parameters.txt',str) 
N4 = int(c4[14].split('=')[1].strip())

In [8]:
n_square = N4**2 # square N in order to interact with oracle

In [9]:
G = Zmod(n_square) # create group ZN^2

In [10]:
def compute_exponent(group,max_iters=15):
    
    """
    compute exponent of group given order of elements for that group
    
    :param group : the group of Z n squared
    :param max_iters : the number of iterations necessary that lambda is constant
    """
    
    sciper = 281141
    expo_lambda = 1
    index = 0
    test = 0
    previous_lambdas = []
    while test<max_iters:
        x = group.random_element() # generate a random element in the group
        msg = str(sciper) + ' ' + str(x)
        u = connect_server("lasecpc28.epfl.ch", int(5556),msg,4)[0][0] # query the oracle to get order of x in ZN^2
        expo_lambda = lcm(expo_lambda,u) # compute lcm of order of element x and current group exponent
        print(test)
        if (len(previous_lambdas) > 0) and (previous_lambdas[index - 1] == expo_lambda): # track that lambda is the same
            test+=1
        else:
            test=0
        previous_lambdas.append(expo_lambda) # list to track changing of lambda
        index+=1

    return previous_lambdas[-1]

In [11]:
exponent = compute_exponent(G)

0
0
0
1
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14


In [12]:
def compute_factors(exponent,group):
    
    """
    computes the factor of a number given the exponent of its group
    
    :param exponent : the exponent of the group
    :param group : group ZN^2
    
    """
    
    s = 0
    t = exponent
    #  first step we try to write exponent as 2^s * t where t is odd
    negative_one = mod(-1,n_square)
    while not is_odd(t): # as t is not odd keep factoring the exponent by 2
        t = t/2
        s+=1
    
    t = Integer(t)
    
    while True:
        x = group.random_element() # generate a random element belonging to the group
        x = power_mod(x,t,n_square) # compute x^t mod N^2
        y = None
        # the aim of this while loop is to keep squaring x until it becomes = 1
        while x != 1:
            y = x
            x = power_mod(x,2,n_square) # square x
        if (y != negative_one) or (y!=None): # found possible group facotrization
            break
    
    return gcd(y-1,n_square)

In [13]:
factor_of_n_square = compute_factors(exponent,G)

In [14]:
# after computing the factor of N^2 we need to get the factors of N
# we do this by computing the gcd of N and the factor of N^2 = p^2,q^2
# this gets us one of the two factors of N
pq = gcd(N4,factor_of_n_square) 

In [16]:
if pq == N4:
    print("The number and the factor are equal")
else:
    print('This implies that the factor of N and the factor of N^2 is either p or q')

This implies that the factor of N and the factor of N^2 is either p or q


In [17]:
# the other factor is N divided by the first factor
other_factor = N4/pq

In [18]:
other_factor = Integer(other_factor)

In [19]:
if pq > other_factor:
    print('Final result = {}'.format(pq))
else:
    print('Final result = {}'.format(other_factor))

Final result = 5123378142938000262688135011468303210515283510676652478427815310822023309782608229300669958883144822713262543221061347870760108944671914114291648097249851474215052894793689792542336442402793799331494335379286027076616774641978295952395786802936898597651053367112668554160521267056726429199965518908560982639


# Diffie-Hellman with a weird group

In [31]:
p5=13154619548672550133712042308539974381709838061251818025210600689125397035760388873249119396788856436034437319159278855793801493143935417084173567495463313
a5=9685474810201602558805724804810549242878718744017144654592998314356661393964437452981013865899226536317607226741948526345300187814509497540390353891752706
u5=584387100862925280952847009177865702031239216921045726765626541175299883520743583629008937861143130229966874329221212518007128814136599242433085098178665
X5=(5156291412594413357456314414998809994710271055170405914913890259077436660133114461906973179337469945302145268246089375739973905081009187368057115813478201, 7216258007008406105981562760746197370726058202661920201399615206908133290763291893650802274928555102731849882636691962576048701747713398237506396439404987)
Y5=(2668794774140752177761604918204396578770762782330123601030627484145234646573913949366635321102627086376590796503784422904981683817155027717506569577184346, 12257683230919185824552769103596922671236749068533075118147634114416205605048004554370310771657192678173914193654210508342628204230831301078271501301496998)

In [32]:
natural_elements = (1,0) # the identity element of G is the identity of Zp* and identity of Zp

In [33]:
# the first pair of the inverse is just u5 inverted (inverse in Zp*) and is equal to one when multiplied with u5
a_inverse = inverse_mod(a5,p5) 
# the second pair should statisfy the equation :  a_inverse * u + a * p =0
# thus the inverse is (a_inverse)^2 * u 
inverse_pair = (a_inverse,mod(-power_mod(a_inverse,2,p5) * u5,p5))

In [34]:
inverse_pair

(9279575166902746037208924802273752618962616775870355155486716724413149939191423996532108604091899138026775157949103892162281614320600769840929170481979800,
 3833615378331589435047235712691739785854761935656702060206453921738491729803010621740112578854527953117919556791361099462949717765281219386522253667849852)

For part c after computing some powers of g, we realize that the following pattern: <br>
g^n = (2^n,2^(n-1) * n * sciper) <br>
Thus we can extract the secret keys by multiplying the second term of the public keys with the inverse of the first term and by 2, then divide by the sciper.

In [35]:
sciper = 281141
inverse_public_x = inverse_mod(X5[0],p5)
secret_key_x = Integer(mod(X5[1] * inverse_public_x * 2,p5))

In [36]:
inverse_public_y = inverse_mod(Y5[0],p5)
secret_key_y = Integer(mod(Y5[1] * inverse_public_y * 2,p5))

In [37]:
# to extract the shared key just multiply secret keys and feed them into the formula
shared_keys = secret_key_x * secret_key_y
shared_secret_key = (power_mod(2,shared_keys,p5),mod(sciper * shared_keys * power_mod(2,shared_keys-1,p5),p5))

In [38]:
shared_secret_key

(5716551393532187100322147027301104131991200394114993203253775687798932419794055735140442303713767719523252007233242889967906280493880346365489220659657633L,
 11382117028100858822009444178533386666330554946493604458195123063966545047380850645880396493147064706070011641698877832773847224687851025328460816729601271)